In [1]:
import pandas as pd
import numpy as np
from scipy import spatial

In [2]:
countries = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

In [3]:
df_gen = pd.read_excel('../cost_curves/data/generation.xlsx', index_col=0)
availability_matrix = np.load('availability_matrix.npy')
df_ptdf = pd.read_excel('./ptdfs.xlsx', index_col=0)

In [4]:
df_substations = pd.read_excel('./substations.xlsx', index_col=0)
df_powerplants = pd.read_excel('./plants.xlsx', index_col=0)
df_grid = pd.read_excel('./grid.xlsx', index_col=0)

substation_locs = []
for index, row in df_substations.iterrows():
    substation_locs.append((row.lon, row.lat))
    
substation_tree = spatial.KDTree(substation_locs)

for index, row in df_powerplants.iterrows():
    result = substation_tree.query([(row.lon,row.lat)])
    df_powerplants.loc[index, 'node'] = result[1][0]
    
df_powerplants.node = df_powerplants.node.astype(int)
display(df_powerplants)

,name,eic,type,zone,installed_capacity,lat,lon,node
0,GDK-Mellach,14W-WGM-KW-----J,natural_gas,AT,838.0,46.911700,15.488300,1066
1,Ottensheim-Wilhering,14W-BOW-KW-----9,hydro_run_of_river,AT,179.0,48.316500,14.151057,1250
2,FHKW Linz Mitte,14WLSG-MITTE---U,natural_gas,AT,216.8,48.299219,14.322935,676
3,Kraftwerk Theiß,14W-KW-THE-EVN-7,natural_gas,AT,485.0,48.394861,15.710147,1040
4,FHKW Linz Süd,14WLSG-1SUED---5,natural_gas,AT,158.0,48.267893,14.346898,676
...,...,...,...,...,...,...,...,...
636,Petrokemija Kutina,NaN,natural_gas,HR,35.0,45.471290,16.792320,1144
637,Dubrovnik,NaN,hydro_reservoir,HR,216.0,42.603001,18.235001,184
638,Fuzine,NaN,hydro_pumped_storage,HR,5.7,45.304985,14.714837,404
639,He Rijeka,NaN,hydro_run_of_river,HR,36.8,45.334999,14.452000,409


In [5]:
observed_lines = df_ptdf.eic.unique()
df_observed_lines = df_grid[df_grid.eic.isin(observed_lines)]
observed_nodes = set(df_observed_lines.substation_1).union(df_observed_lines.substation_2)
df_substations = df_substations[df_substations.index.isin(observed_nodes)]

In [7]:
# GSK1 zonal denominator

gsk1_matrix = np.zeros((len(df_gen), len(df_substations)))

gsk1_num_plants_z = {}
for zone in countries:
    print(zone)
    
    plants_z = df_powerplants[df_powerplants.zone == zone]
    num_plants_z[zone] = np.zeros(len(df_gen))
    for t, (t_index, gen) in enumerate(df_gen.iterrows()):
        sum_plants_z = 0
        for p, (p_index, row) in enumerate(plants_z.iterrows()):
            if availability_matrix[t][p] > 0:
                sum_plants_z += 1
                
        gsk1_num_plants_z[zone][t] = sum_plants_z

AT
BE
CZ
DE_LU
FR
HR
HU
NL
PL
RO
SI
SK


In [15]:
# GSK1

for n, (n_index, substation) in enumerate(df_substations.iterrows()):
    print(n)
    zone = df_substations.loc[n_index, 'zone']
    if zone == 'DE':
        zone = 'DE_LU'
    plants_n = df_powerplants[df_powerplants.node == n_index]
        
    for t, (t_index, gen) in enumerate(df_gen.iterrows()):
       
        num_plants_n = 0
        for p, (p_index, row) in enumerate(plants_n.iterrows()):
            if availability_matrix[t][p] > 0:
                num_plants_n += 1
                
        gsk1_matrix[t][n] = num_plants_n/gsk1_num_plants_z[zone][t] if gsk1_num_plants_z[zone][t] > 0 else 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
display(gsk1_matrix)

with open('gsk1_matrix.npy', 'wb') as f:
    np.save(f, gsk1_matrix)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
# GSK2 zonal denominator

gsk2_matrix = np.zeros((len(df_gen), len(df_substations)))
flex_types = [
    'coal_gas', 'natural_gas', 'oil', 'oil_shale', 'peat', 'hydro_pumped_storage'
]

gsk2_num_plants_z = {}
for zone in countries:
    print(zone)
    
    plants_z = df_powerplants[(df_powerplants.zone == zone) & (df_powerplants.type.isin(flex_types))]
    gsk2_num_plants_z[zone] = np.zeros(len(df_gen))
    for t, (t_index, gen) in enumerate(df_gen.iterrows()):
        sum_plants_z = 0
        for p, (p_index, row) in enumerate(plants_z.iterrows()):
            if availability_matrix[t][p] > 0:
                sum_plants_z += 1
                
        gsk2_num_plants_z[zone][t] = sum_plants_z

AT
BE
CZ
DE_LU
FR
HR
HU
NL
PL
RO
SI
SK


In [23]:
# GSK2

for n, (n_index, substation) in enumerate(df_substations.iterrows()):
    print(n)
    zone = df_substations.loc[n_index, 'zone']
    if zone == 'DE':
        zone = 'DE_LU'
    plants_n = df_powerplants[(df_powerplants.node == n_index) & (df_powerplants.type.isin(flex_types))]
        
    for t, (t_index, gen) in enumerate(df_gen.iterrows()):
       
        num_plants_n = 0
        for p, (p_index, row) in enumerate(plants_n.iterrows()):
            if availability_matrix[t][p] > 0:
                num_plants_n += 1
                
        gsk2_matrix[t][n] = num_plants_n/gsk2_num_plants_z[zone][t] if gsk2_num_plants_z[zone][t] > 0 else 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
display(gsk2_matrix)
print(np.mean(np.mean(gsk2_matrix)))

with open('gsk2_matrix.npy', 'wb') as f:
    np.save(f, gsk2_matrix)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

0.004400963684647565


In [25]:
# GSK3 zonal denominator

gsk3_matrix = np.zeros((len(df_gen), len(df_substations)))

gsk3_p_plants_z = {}
for zone in countries:
    print(zone)
    
    plants_z = df_powerplants[df_powerplants.zone == zone]
    gsk3_p_plants_z[zone] = np.zeros(len(df_gen))
    for t, (t_index, gen) in enumerate(df_gen.iterrows()):
        sum_plants_z = 0
        for p, (p_index, row) in enumerate(plants_z.iterrows()):
            if availability_matrix[t][p] > 0:
                sum_plants_z += availability_matrix[t][p]
                
        gsk3_p_plants_z[zone][t] = sum_plants_z

AT
BE
CZ
DE_LU
FR
HR
HU
NL
PL
RO
SI
SK


In [26]:
# GSK3

for n, (n_index, substation) in enumerate(df_substations.iterrows()):
    print(n)
    zone = df_substations.loc[n_index, 'zone']
    if zone == 'DE':
        zone = 'DE_LU'
    plants_n = df_powerplants[df_powerplants.node == n_index]
        
    for t, (t_index, gen) in enumerate(df_gen.iterrows()):
       
        p_plants_n = 0
        for p, (p_index, row) in enumerate(plants_n.iterrows()):
            if availability_matrix[t][p] > 0:
                p_plants_n += availability_matrix[t][p]
                
        gsk3_matrix[t][n] = p_plants_n/gsk3_p_plants_z[zone][t] if gsk3_p_plants_z[zone][t] > 0 else 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [27]:
display(gsk3_matrix)
print(np.mean(np.mean(gsk3_matrix)))

with open('gsk3_matrix.npy', 'wb') as f:
    np.save(f, gsk3_matrix)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

0.007702434187727182
